### Remove mixed pixels

In [ ]:
inputs = [{
    'osmlulc'  : '/home/gisuser/autocls2/ualc_2211.gpkg',
    'osmlyr'   : 'ua18_coimbra_f0',
    'lulc_col' : 'classid',
    'refimg'   : '/home/gisuser/autocls2/refimgs/ref_coimbra.tif',
    'out'      : '/home/gisuser/autocls2/uafilter/ua18_coimbra_f1.tif'
}, {
    'osmlulc'  : '/home/gisuser/autocls2/ualc_2211.gpkg',
    'osmlyr'   : 'ua18_faro_f0',
    'lulc_col' : 'classid',
    'refimg'   : '/home/gisuser/autocls2/refimgs/ref_faro.tif',
    'out'      : '/home/gisuser/autocls2/uafilter/ua18_faro_f1.tif'
}, {
    'osmlulc'  : '/home/gisuser/autocls2/ualc_2211.gpkg',
    'osmlyr'   : 'ua18_porto_f0',
    'lulc_col' : 'classid',
    'refimg'   : '/home/gisuser/autocls2/refimgs/ref_porto.tif',
    'out'      : '/home/gisuser/autocls2/uafilter/ua18_porto_f1.tif'
}, {
    'osmlulc'  : '/home/gisuser/autocls2/ualc_2211.gpkg',
    'osmlyr'   : 'ua18_lisboa_f0',
    'lulc_col' : 'classid',
    'refimg'   : '/home/gisuser/autocls2/refimgs/ref_lisboa.tif',
    'out'      : '/home/gisuser/autocls2/uafilter/ua18_lisboa_f1.tif'
}]

from glass.ete.autocls.osmfilter import rm_mixed_pixels

for d in inputs:
    rm_mixed_pixels(
        d['osmlulc'], d['osmlyr'], d['lulc_col'],
        d['refimg'], d['out']
    )